# 🕸️ LangGraph - ¿Qué lo hace especial?
## Un ejemplo que SOLO LangGraph puede hacer

### 🎯 El Problema:

Imagina que quieres un **agente que pueda autocorregirse**:

```
1. El agente genera una respuesta
2. Un verificador revisa si la respuesta es correcta
3. Si está MAL, el agente lo intenta de nuevo (LOOP)
4. Máximo 3 intentos
5. Si falla todo, pide ayuda humana
```

### ❌ Con LangChain clásico:
- **Imposible** crear loops condicionales naturalmente
- Tendrías que escribir código manual complejo
- No hay forma de "volver atrás" en el flujo

### ✅ Con LangGraph:
- **Natural** - Es un grafo, puede tener ciclos
- **Simple** - Defines el flujo visualmente
- **Poderoso** - Control total de cuándo repetir

## 📦 Instalación

In [ ]:
!pip install langgraph langchain-openai langchain-core python-dotenv -q

## ⚙️ Configuración

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("⚠️ OPENAI_API_KEY no encontrada")

print("✅ Configuración lista")

---
# 🎯 El Ejemplo: Agente que se Autocorrige

Vamos a crear un agente que:
1. **Resuelve** un problema matemático
2. **Verifica** su propia respuesta
3. **Se corrige** si está mal (máximo 3 intentos)
4. **Pide ayuda** si no puede resolverlo

## Flujo del Grafo:

```
   START
     │
     ▼
┌──────────┐
│ RESOLVER │ ──────┐
└──────────┘       │
     │             │
     ▼             │
┌──────────┐       │
│VERIFICAR │       │ LOOP (si está mal)
└──────────┘       │
     │             │
     ├─ ✅ Correcto → END
     │
     ├─ ❌ Incorrecto (intentos < 3) ──┘
     │
     └─ ❌ Incorrecto (intentos = 3) → PEDIR_AYUDA → END
```

## Paso 1: Definir el State

In [ ]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# State que pasa entre nodos
class State(TypedDict):
    problema: str           # El problema a resolver
    respuesta: str          # La respuesta actual
    es_correcta: bool       # ¿La respuesta es correcta?
    intentos: int           # Número de intentos
    feedback: str           # Feedback del verificador
    necesita_ayuda: bool    # Si necesita ayuda humana

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ State definido")

## Paso 2: Definir los Nodos (funciones)

In [ ]:
def resolver_problema(state: State) -> State:
    """Nodo 1: El agente intenta resolver el problema"""
    
    print(f"\n🤖 Intento #{state['intentos'] + 1}")
    
    # Prompt con feedback previo si existe
    if state["feedback"]:
        prompt = f"""Problema: {state['problema']}

Tu respuesta anterior estaba INCORRECTA.
Feedback: {state['feedback']}

Intenta de nuevo. Piensa paso a paso y revisa tu matemática.
Responde SOLO con el número final."""
    else:
        prompt = f"""Resuelve este problema matemático:
{state['problema']}

Piensa paso a paso.
Responde SOLO con el número final."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    respuesta = response.content.strip()
    
    print(f"   Respuesta: {respuesta}")
    
    return {
        **state,
        "respuesta": respuesta,
        "intentos": state["intentos"] + 1,
        "feedback": ""  # Limpiar feedback anterior
    }

def verificar_respuesta(state: State) -> State:
    """Nodo 2: Verificador revisa si la respuesta es correcta"""
    
    print("   🔍 Verificando...")
    
    prompt = f"""Eres un verificador matemático estricto.

Problema: {state['problema']}
Respuesta del estudiante: {state['respuesta']}

¿La respuesta es correcta? Responde con:
- "CORRECTO" si es correcta
- "INCORRECTO: [explicación del error]" si es incorrecta

Sé específico en el error."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    verificacion = response.content.strip()
    
    es_correcta = verificacion.startswith("CORRECTO")
    
    if es_correcta:
        print("   ✅ CORRECTO!")
    else:
        print(f"   ❌ INCORRECTO: {verificacion}")
    
    return {
        **state,
        "es_correcta": es_correcta,
        "feedback": verificacion if not es_correcta else ""
    }

def pedir_ayuda(state: State) -> State:
    """Nodo 3: Si falló 3 veces, pide ayuda humana"""
    
    print("\n🆘 PIDIENDO AYUDA HUMANA")
    print(f"   El agente no pudo resolver el problema después de {state['intentos']} intentos")
    print(f"   Última respuesta: {state['respuesta']}")
    print(f"   Feedback: {state['feedback']}")
    
    return {
        **state,
        "necesita_ayuda": True
    }

print("✅ Nodos definidos")

## Paso 3: Función de Decisión (Routing)

In [ ]:
def decidir_siguiente_paso(state: State) -> Literal["resolver", "pedir_ayuda", "end"]:
    """Decide qué hacer después de verificar"""
    
    # Si es correcta, terminamos
    if state["es_correcta"]:
        return "end"
    
    # Si ya intentó 3 veces, pide ayuda
    if state["intentos"] >= 3:
        return "pedir_ayuda"
    
    # Si no, intenta de nuevo (LOOP)
    return "resolver"

print("✅ Routing definido")

## Paso 4: Construir el Grafo

In [ ]:
# Crear el grafo
workflow = StateGraph(State)

# Agregar nodos
workflow.add_node("resolver", resolver_problema)
workflow.add_node("verificar", verificar_respuesta)
workflow.add_node("pedir_ayuda", pedir_ayuda)

# Definir flujo
workflow.add_edge(START, "resolver")
workflow.add_edge("resolver", "verificar")

# Edge CONDICIONAL desde verificar
workflow.add_conditional_edges(
    "verificar",
    decidir_siguiente_paso,
    {
        "resolver": "resolver",      # LOOP: volver a intentar
        "pedir_ayuda": "pedir_ayuda",
        "end": END
    }
)

workflow.add_edge("pedir_ayuda", END)

# Compilar
app = workflow.compile()

print("✅ Grafo construido")
print("\n🕸️ Estructura:")
print("   START → resolver → verificar")
print("                         │")
print("                         ├─ ✅ correcto → END")
print("                         ├─ ❌ incorrecto (< 3 intentos) → resolver (LOOP)")
print("                         └─ ❌ incorrecto (3 intentos) → pedir_ayuda → END")

---
# 🎬 DEMO - Ver el Grafo en Acción

Vamos a probar con 2 problemas:
1. **Fácil**: Que el agente pueda resolver
2. **Difícil**: Que falle y pida ayuda

## Demo 1: Problema Fácil (con trampa)

In [ ]:
print("\n" + "="*70)
print("🎯 DEMO 1: Problema con trampa")
print("="*70)

problema1 = """Si tienes 3 manzanas y compras 5 más, pero luego regalas 2.
¿Cuántas manzanas tienes ahora?"""

print(f"\n📝 Problema: {problema1}")
print("\n💡 Respuesta correcta: 6 manzanas (3 + 5 - 2 = 6)\n")

resultado = app.invoke({
    "problema": problema1,
    "respuesta": "",
    "es_correcta": False,
    "intentos": 0,
    "feedback": "",
    "necesita_ayuda": False
})

print("\n" + "="*70)
print("📊 RESULTADO FINAL")
print("="*70)
print(f"Respuesta final: {resultado['respuesta']}")
print(f"¿Es correcta?: {resultado['es_correcta']}")
print(f"Intentos usados: {resultado['intentos']}")
print(f"¿Necesitó ayuda?: {resultado['necesita_ayuda']}")

## Demo 2: Problema Difícil (Intencionalmente complicado)

In [ ]:
print("\n" + "="*70)
print("🎯 DEMO 2: Problema difícil (para ver el loop y pedir ayuda)")
print("="*70)

# Problema con trampa lógica
problema2 = """Un granjero tiene 17 ovejas. Todas menos 9 se mueren.
¿Cuántas ovejas le quedan vivas?"""

print(f"\n📝 Problema: {problema2}")
print("\n💡 Respuesta correcta: 9 ovejas (TODAS MENOS 9 se mueren = quedan 9)")
print("   ⚠️ Trampa común: Pensar que son 17-9=8\n")

resultado2 = app.invoke({
    "problema": problema2,
    "respuesta": "",
    "es_correcta": False,
    "intentos": 0,
    "feedback": "",
    "necesita_ayuda": False
})

print("\n" + "="*70)
print("📊 RESULTADO FINAL")
print("="*70)
print(f"Respuesta final: {resultado2['respuesta']}")
print(f"¿Es correcta?: {resultado2['es_correcta']}")
print(f"Intentos usados: {resultado2['intentos']}")
print(f"¿Necesitó ayuda?: {resultado2['necesita_ayuda']}")

---
# 🎓 ¿Por qué esto es ESPECIAL?

## ❌ Intenta hacer esto con LangChain clásico:

```python
# Con langchain-classic NO PUEDES:

# 1. Crear LOOPS condicionales naturalmente
chain = prompt | llm | parser
# ↑ Es lineal, no puede "volver atrás"

# 2. Decidir dinámicamente el siguiente paso
# 3. Contar intentos automáticamente
# 4. Tener múltiples caminos que convergen

# Tendrías que escribir un while loop manual:
intentos = 0
while intentos < 3:
    respuesta = chain.invoke(...)
    if verificar(respuesta):
        break
    intentos += 1
# ↑ Código feo, difícil de mantener, sin visualización
```

## ✅ Con LangGraph:

```python
# DECLARAS el flujo como un GRAFO
workflow.add_conditional_edges(
    "verificar",
    decidir_siguiente_paso,
    {
        "resolver": "resolver",      # ← LOOP automático
        "pedir_ayuda": "pedir_ayuda",
        "end": END
    }
)
# ↑ Claro, visual, mantenible
```

---

## 🔥 Otros casos de uso ÚNICOS de LangGraph:

1. **Multi-agent con comunicación bidireccional**
   - Agente A consulta a Agente B
   - Agente B responde
   - Agente A decide qué hacer

2. **Human-in-the-loop en puntos específicos**
   - Pausa el flujo
   - Espera aprobación humana
   - Continúa o cancela

3. **Retry con estrategias diferentes**
   - Intento 1: Método A
   - Intento 2: Método B
   - Intento 3: Método C

4. **Workflows complejos de negocio**
   - Aprobaciones multi-nivel
   - Escalamiento automático
   - Rollback si falla

---

## 💡 Conclusión:

### LangGraph es para cuando necesitas:
- ✅ **Loops** y **ciclos**
- ✅ **Decisiones** complejas
- ✅ **Múltiples caminos** que se cruzan
- ✅ **Estado compartido** entre nodos
- ✅ **Control total** del flujo

### LangChain clásico/LCEL es para:
- ✅ Flujos **lineales** simples
- ✅ Pipelines de **transformación de datos**
- ✅ Chains que **no necesitan loops**

---

**¿Preguntas?** 🤔